<a href="https://colab.research.google.com/github/ggzlemos/busca_informada/blob/main/busca_informada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [16]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

## Busca com Informação

In [1]:
import numpy as np

In [9]:
class Nodo:
  def __init__(self, estado):       
    self.estado = estado
    self.caminho = []
    self.pai = None
    self.g = 0
    self.h = 0
    self.f = 0

  def getEstado(self):
    return self.estado

  def setPai(self, nodo):
    self.pai = nodo

  def getPai(self):
    return self.pai  
   
  def coordVazio(self):
    i, j = np.where(self.estado == 0)
    return i[0], j[0]

  def constroiCaminho(self):
    caminho = []
    filho = self
    pai = self.getPai()
    while pai != None:
      caminho.append(pai.getEstado())
      pai = pai.getPai()    
    caminho.reverse()
    caminho.append(self.estado)
    self.caminho = caminho

  def imprimeCaminho(self):
    self.constroiCaminho()
    print("Resultado:")
    for i in self.caminho:
      print(i)
    
  def getProfundidade(self):
    return len(self.caminho)-1

  def getDim(self):
    return self.estado.shape

  def geraEstados(self):
    vizinhos = []  
    I, J = self.getDim()
    i, j = self.coordVazio() 
    dj = np.array([-1, 1, 0, 0])
    di = np.array([0, 0, 1, -1])
        
    for k in range(0, 4):
      i_novo = i + di[k]
      j_novo = j + dj[k]  
      if not i_novo < 0 and not j_novo < 0:
        if not i_novo >= I and not j_novo >= J:
          novo_vizinho = self.estado.copy()       
          novo_vizinho[i][j], novo_vizinho[i_novo][j_novo] = novo_vizinho[i_novo][j_novo], novo_vizinho[i][j]
          novo_vizinho = Nodo(novo_vizinho)        
          vizinhos.append(novo_vizinho)                  
    return vizinhos
  
  def calculaG(self ):
    if self.pai == None:
      self.g = 0 
    else:
      self.g = self.pai.g + 1
    return self.g

  def atualizaG(self, estado):
    self.g = estado.g + 1 

  def calculaH(self, estado_final):
    a = self.estado.reshape(-1)
    b = estado_final.reshape(-1)
    self.h = (a != b).sum() -1
    return self.h

  def calculaF(self):     
    self.f = self.g + self.h

  def getF(self):
    return self.f

In [3]:
def foi_visitado(visitados, nodo):
  estado = nodo.getEstado()
  for i in visitados:
    if np.array_equal(i.getEstado(), estado):
      return True
  return False

In [15]:
def menorF(fila):
  dif = float('inf')
  menor = None
  for estado in fila:
    if estado.f < dif:
      dif = estado.f
      menor = estado
  fila.remove(menor)    
  return menor

In [5]:
def a_estrela(estado_inicial, estado_final):
  fila = [estado_inicial]
  visitados = [] 
 
  while len(fila) > 0:   
    nodo = menorF(fila)
    visitados.append(nodo)

    if np.array_equal(nodo.getEstado(), estado_final):             
      return nodo

    vizinhanca = nodo.geraEstados()
    for vizinho in vizinhanca:
      if not foi_visitado(visitados, vizinho):
        if not foi_visitado(fila, vizinho):
          vizinho.setPai(nodo)
          vizinho.calculaG()
          vizinho.calculaH(estado_final)
          vizinho.calculaF()
          fila.append(vizinho)
        elif foi_visitado(fila, vizinho):
          if vizinho.g > nodo.g+1:
            vizinho.atualizaG(nodo)
            vizinho.calculaF()
  return Nodo(np.array([])) #retorna um Nodo vazio se não encontrar um caminho               

In [13]:
estado_inicial = Nodo(np.array([1,2,3,4,5,6,0,7,8]).reshape((3,3)))
estado_final = np.array([1,2,3,4,5,6,7,8,0]).reshape((3,3))
resultado = a_estrela(estado_inicial, estado_final)
resultado.imprimeCaminho()
resultado.getProfundidade()

Resultado:
[[1 2 3]
 [4 5 6]
 [0 7 8]]
[[1 2 3]
 [4 5 6]
 [7 0 8]]
[[1 2 3]
 [4 5 6]
 [7 8 0]]


2

In [14]:
estado_inicial = Nodo(np.array([0,1,5,3,8,2,7,4,6]).reshape((3,3)))
estado_final = np.array([1,2,3,4,5,6,7,8,0]).reshape((3,3))
resultado = a_estrela(estado_inicial, estado_final)
resultado.imprimeCaminho()
resultado.getProfundidade()

Resultado:
[[0 1 5]
 [3 8 2]
 [7 4 6]]
[[1 0 5]
 [3 8 2]
 [7 4 6]]
[[1 5 0]
 [3 8 2]
 [7 4 6]]
[[1 5 2]
 [3 8 0]
 [7 4 6]]
[[1 5 2]
 [3 8 6]
 [7 4 0]]
[[1 5 2]
 [3 8 6]
 [7 0 4]]
[[1 5 2]
 [3 0 6]
 [7 8 4]]
[[1 5 2]
 [0 3 6]
 [7 8 4]]
[[1 5 2]
 [7 3 6]
 [0 8 4]]
[[1 5 2]
 [7 3 6]
 [8 0 4]]
[[1 5 2]
 [7 3 6]
 [8 4 0]]
[[1 5 2]
 [7 3 0]
 [8 4 6]]
[[1 5 2]
 [7 0 3]
 [8 4 6]]
[[1 5 2]
 [7 4 3]
 [8 0 6]]
[[1 5 2]
 [7 4 3]
 [0 8 6]]
[[1 5 2]
 [0 4 3]
 [7 8 6]]
[[1 5 2]
 [4 0 3]
 [7 8 6]]
[[1 0 2]
 [4 5 3]
 [7 8 6]]
[[1 2 0]
 [4 5 3]
 [7 8 6]]
[[1 2 3]
 [4 5 0]
 [7 8 6]]
[[1 2 3]
 [4 5 6]
 [7 8 0]]


20

## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento


###O A* se mostrou mais eficiente em resolver o problema do quebra-cabeça deslizante. Ele executa em tempo consideravelmente menor que a busca em largura e a busca em profundidade.

###Em relação ao consumo de memória, o algoritmo não estourou a memória em nenhum dos exemplos testados, como havia ocorrido na execução da busca em largura, por exemplo. Porém , ele também armazena na memória  outros nodos que não fazem parte do caminho até a resposta.